In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
import torch

from utils.utilities_DL import get_MultiModel_loss_args_emb_opts,load_init_trainer,load_prediction
from trainer import Trainer
from constants.config import get_args,update_modif
from constants.paths import folder_path,file_name,SAVE_DIRECTORY

from HP_tuning.ray_search_space import get_search_space_ray 
from HP_tuning.ray_config import get_ray_config
import ray 
from ray import tune 
import os 

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

'pynvml' is not available on this environment.


In [3]:
from high_level_DL_method import load_everything 
from utils.utilities_DL import match_period_coverage_with_netmob,get_small_ds

# Load config
model_name = 'STGCN' #'CNN'
args = get_args(model_name)

# Modification : 
args.K_fold = 5
args.ray = True
args.W = 0  # IMPORTANT AVEC NETMOB
args.epochs = 1
args.loss_function_type = 'MSE' # 'quantile'
args.mixed_precision = False
args.torch_compile = False

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(file_name)
(coverage,args) = get_small_ds(small_ds,coverage,args)

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'ImageAvgPooling'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

Model: STGCN, K_fold = 5
!!! Loss function: MSE 


In [6]:
from build_inputs.load_subway_in import load_subway_in
from build_inputs.load_preprocessed_dataset import load_complete_ds
from dataset import TensorDataset


def add_U_test_and_Utarget_test(subway_ds_tmps,subway_ds):
    ''' Tackle U_test and Utarget_test'''
    U_test_tmps = TensorDataset(subway_ds.U_test, normalized = False, normalizer=subway_ds_tmps.normalizer)
    U_test_tmps.normalize(feature_vect = True)

    Utarget_test_tmps = TensorDataset(subway_ds.Utarget_test, normalized = False, normalizer=subway_ds_tmps.normalizer)
    Utarget_test_tmps.normalize(feature_vect = True)

    subway_ds_tmps.U_test = U_test_tmps.tensor
    subway_ds_tmps.Utarget_test = Utarget_test_tmps.tensor
    # ...
    return (subway_ds_tmps)

def add_contextual_U_test(subway_ds_tmps,subway_ds,NetMob_ds_tmps):
    ''' Tackle contextual Test vector:''' 
    for name in subway_ds.contextual_tensors.keys():

        if name == 'netmob':
            U_context_tmps = TensorDataset(subway_ds.contextual_tensors[name]['test'], normalized = False, normalizer=NetMob_ds_tmps.normalizer)
            U_context_tmps.normalize(feature_vect = True)
            subway_ds_tmps.contextual_tensors[name]['test'] = U_context_tmps.tensor
        elif 'calendar' in name:
            subway_ds_tmps.contextual_tensors[name]['test'] = subway_ds.contextual_tensors[name]['test'] 
        else:
            raise NotImplementedError(f'contextual data {name} has not been implemented')

    return(subway_ds_tmps)

# ========================================
# ===== ===== SPLIT K FOLD ===== =====


#(subway_ds,dataset,invalid_dates) = load_subway_in(file_name,args,coverage,normalize=False)
subway_ds,NetMob_ds,positions,args,args_vision,args_embedding,dic_class2rpz = load_complete_ds(dataset_names,args,coverage,folder_path,file_name,vision_model_name, normalize = False)
print(f"U_test: {subway_ds.U_test.shape}\nCalendar1: {subway_ds.contextual_tensors['calendar_1']['test'].shape} \nNetmob-test: {subway_ds.contextual_tensors['netmob']['test'].shape}")
print('None of these tensor have been normalized.')

K_subway_ds = []
# Remove test-dates:
first_test_date = subway_ds.tensor_limits_keeper.first_test_date
coverage_without_test =  coverage[coverage<first_test_date]
n = len(coverage_without_test)
# ... 

# Adapt Valid and Train Prop (cause we want Test_prop = 0)
train_prop_tmps = args.train_prop/(args.train_prop+args.valid_prop)
valid_prop_tmps = args.valid_prop/(args.train_prop+args.valid_prop)

args.train_prop = train_prop_tmps
args.valid_prop = valid_prop_tmps
args.test_prop = 0
# ...

# Découpe la dataframe en K_fold 
for k in range(args.K_fold):
    # Slicing 
    print(f'\nFold n°{k}')
    if args.validation == 'sliding_window':
        width_dataset = int(n/(1+(args.K_fold-1)*valid_prop_tmps))   # Stay constant. W = N/(1 + (K-1)*Pv/(Pv+Pt))
        l_lim_pos = int(k*valid_prop_tmps*width_dataset)    # Shifting of (valid_prop/train_prop)% of the width of the window, at each iteration 
        if k == args.K_fold - 1:
            u_lim_pos = n
        else:
            u_lim_pos = l_lim_pos + width_dataset
        
        # Get df_tmps and coverage_tmps:
        coverage_tmps = coverage_without_test[l_lim_pos:u_lim_pos] 

    subway_ds_tmps,NetMob_ds_tmps,_,_,_,_,_ = load_complete_ds(dataset_names,args,coverage_tmps,folder_path,file_name,vision_model_name, normalize = True)

    # Tackle U_test and Utarget_test (normalize U_test with normalizer from subway_ds_TMPS):
    subway_ds_tmps = add_U_test_and_Utarget_test(subway_ds_tmps,subway_ds)

    # Tackle contextual Test vector (normalize U_test_contextual with normalizer from NetMob_ds_TMPS):
    subway_ds_tmps = add_contextual_U_test(subway_ds_tmps,subway_ds,NetMob_ds_tmps)

    # Tackle dataloader: load dataloader again (call it 2 times so longer but easier to implement)
    subway_ds_tmps.get_dataloader()
    # ...
    

    K_subway_ds.append(subway_ds_tmps)

# ========================================
# ===== ===== SPLIT K FOLD ===== =====


Fold n°0
Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-04-16 03:45:00

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

U size:  torch.Size([2896, 40, 7]) Utarget size:  torch.Size([2896, 40, 1])
U_train size:  torch.Size([2177, 40, 7]) Utarget_train size:  torch.Size([2177, 40, 1])
U_valid size:  torch.Size([622, 40, 7]) Utarget_valid size:  torch.Size([622, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(1.)
U_valid min:  tensor(0.) U_valid max:  tensor(1.1906)
Load des données NetMob .pt impossible. Création d'un random Tensor
Init NetMob Dataset:  torch.Size([2992, 40, 2, 8, 8])
Number of Nan Value:  

In [40]:
len(K_subway_ds)

5

In [17]:
subway_ds.warning()
print(f'Compute statistics through dimensions: {subway_ds.dims} ')
Datasets,DataLoader_list = [],[]

Compute statistics through dimensions: [0] 


In [ ]:
dataset_init = subway_ds.clean_dataset_get_tensor_and_train_valid_test_split(self.df,subway_ds.invalid_dates,args.train_prop,args.valid_prop,args.test_prop, normalize = False)

In [ ]:
dataset_init = 
dataset_init = split_normalize_load_feature_vect(dataset_init,invalid_dates,train_prop,valid_prop,test_prop,normalize = False)

In [ ]:

# Fait la 'Hold-Out' séparation, pour enlever les dernier mois de TesT
df_hold_out = self.df[: dataset_init.first_test_date]  



# Récupère la Taille de cette DataFrame
n = len(df_hold_out)

# Adapt Valid and Train Prop (cause we want Test_prop = 0)
train_prop_tmps = args.train_prop/(args.train_prop+args.valid_prop)
valid_prop_tmps = args.valid_prop/(args.train_prop+args.valid_prop)

args.train_prop = train_prop_tmps
args.valid_prop = valid_prop_tmps

# Découpe la dataframe en K_fold 
for k in range(args.K_fold):
    # Slicing 
    if args.validation == 'sliding_window':
        width_dataset = int(n/(1+(args.K_fold-1)*valid_prop_tmps))   # Stay constant. W = N/(1 + (K-1)*Pv/(Pv+Pt))
        l_lim_pos = int(k*valid_prop_tmps*width_dataset)    # Shifting of (valid_prop/train_prop)% of the width of the window, at each iteration 
        if k == args.K_fold - 1:
            u_lim_pos = n
        else:
            u_lim_pos = l_lim_pos + width_dataset

        # Get df_tmps and coverage_tmps:
        coverage_tmps = coverage[l_lim_pos:u_lim_pos] 
        df_tmps = df_hold_out[l_lim_pos:u_lim_pos]       

    # Traditionnal pre-process : 
    dataset_tmps = DataSet(df_tmps, Weeks = self.Weeks, Days = self.Days, historical_len= self.historical_len,
                            step_ahead=self.step_ahead,time_step_per_hour=self.time_step_per_hour)
    subway_ds_tmps = preprocess_subway_in(dataset_tmps,args,invalid_dates)   
    subway_ds_tmps,positions,args,args_vision,args_embedding,dic_class2rpz = load_complete_ds(dataset_names,args,coverage_tmps,folder_path,file_name,vision_model_name,subway_ds = subway_ds_tmps, dataset = dataset_tmps, invalid_dates = invalid_dates)
    # ...   



In [4]:
model,subway_ds,loss_function,optimizer,scheduler,args,args_embedding,args_vision,positions,dic_class2rpz = load_everything(dataset_names,folder_path,file_name,args,coverage,vision_model_name)

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Training Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Validation Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

 Tackling Testing Set
Values with issues:  0.000%
Regular Values that we have to set to 0:  0.000%

U size:  torch.Size([5662, 40, 8]) Utarget size:  torch.Size([5662, 40, 1])
U_train size:  torch.Size([2934, 40, 8]) Utarget_train size:  torch.Size([2934, 40, 1])
U_valid size:  torch.Size([978, 40, 8]) Utarget_valid size:  torch.Size([978, 40, 1])
U_test size:  torch.Size([979, 40, 8]) Utarget_test size: 

In [ ]:
## Hyper Parameter Tuning sur le Fold 0
def load_trainer(config,folder_path,file_name,args):

    for key, value in config.items():
        if hasattr(args, key):
            setattr(args, key, value)

    Datasets,DataLoader_list,dic_class2rpz,nb_words_embedding,time_slots_labels,dic_rpz2class = load_init_trainer(folder_path,file_name,args)
    (loss_function,Model_list,Optimizer_list,Scheduler_list,args_embedding) = get_MultiModel_loss_args_emb_opts(args,nb_words_embedding,dic_class2rpz,n_vertex = len(Datasets[0].columns))
    dataset,dataloader,model,optimizer,scheduler = Datasets[0],DataLoader_list[0],Model_list[0],Optimizer_list[0],Scheduler_list[0]


    trainer = Trainer(dataset,model,dataloader,
                    args,optimizer,loss_function,scheduler = scheduler,
                    args_embedding=args_embedding,dic_class2rpz=dic_class2rpz)
    return(trainer)

def Train_with_tune(config,folder_path,file_name,args):
    trainer = load_trainer(config,folder_path,file_name,args)
    result_df = trainer.train_and_valid()

    
def run_tuning_and_save_results(args,num_samples):
    config = get_search_space_ray(args)
    ray_scheduler,ray_search_alg,resources_per_trial,num_gpus,max_concurrent_trials,num_cpus = get_ray_config(args)
    
    def trainer(config):
        return(Train_with_tune(config,folder_path,file_name,args))
        
    if ray.is_initialized:
        ray.shutdown()
        ray.init(num_gpus=num_gpus,num_cpus=num_cpus)

        
    analysis = tune.run(
            trainer,
            config=config,
            num_samples=num_samples,  # Increase num_samples for more random combinations
            resources_per_trial = resources_per_trial,
            max_concurrent_trials = max_concurrent_trials,
            scheduler = ray_scheduler,
            search_alg = ray_search_alg,
        )

    name_save = f"HyperparameterTuning/Htuning_ray_analysis_{args.model_name}_loss{args.loss_function_type}_TE_{args.time_embedding}"
    analysis.results_df.to_csv(f'{name_save}.csv')

In [ ]:
# ==== GET PARAMETERS ====
# Load config
model_name = 'STGCN'  #'CNN'
args = get_args(model_name)

# Classic Modification : 
args.epochs = 1
args.loss_function_type = 'MSE' # 'quantile'
# ...

# Modification pour HyperParameterTuning:
args.K_fold = 5
args.ray = True
# ...

# Update Modif 
args = update_modif(args,name_gpu='cuda:0')
# ...

run_tuning_and_save_results(args,num_samples=1000)